In [ ]:
# Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import imdb

In [ ]:
# load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
# Load the pre-trained model
model = load_model('RNN_model.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,939,077 (15.03 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,626,052 (10.02 MB)

In [9]:
# Function to decode an encoded review
def decode_review(encoded_review, reverse_word_index):
    """
    Decodes a numerical review back into text using reverse_word_index.
    Adjusts for reserved indices by subtracting 3.
    """
    return " ".join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input for model prediction
def preprocess_text(text, word_index, maxlen=500, vocab_size=10000):
    """
    Converts raw text into a padded sequence of integers for model input.
    Unknown words are mapped to index 2 (typically 'UNK').
    Limits vocabulary size to vocab_size.
    """
    # Lowercase and split the text
    words = text.lower().split()
    # Convert words to integer indices, limiting to vocab_size
    encoded = [word_index.get(word, 2) for word in words if word_index.get(word, 2) < vocab_size]  # 2 = unknown token, filter by vocab_size
    # Pad the sequence to the desired length
    padded = sequence.pad_sequences([encoded], maxlen=maxlen, padding='post', truncating='post')
    return padded

In [15]:
# Prediction function
def predict_sentiment(review):
  preprocess_input = preprocess_text(review, word_index)
  prediction = model.predict(preprocess_input)
  sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"
  return sentiment, prediction[0][0]

In [16]:
# User input and prediction
user_review = input("Enter a movie review: ")
sentiment, score = predict_sentiment(user_review)
print(f"Review: {user_review}")
print(f"Predicted Sentiment: {sentiment}")
print(f"Prediction Score: {score}")

Enter a movie review: movie was fantastic
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Review: movie was fantastic
Predicted Sentiment: Negative
Prediction Score: 0.19699877500534058
